In [4]:
from pyspark.sql import SparkSession

nyc_taxi_challenge_path = '../../nyc-taxi-challenge/'

spark = SparkSession.builder.appName("NYC Taxi Challenge").getOrCreate()
spark

# Exploring the datasets

## Trips from 2009

This dataset has informations about taxi trips of 2009

In [7]:
json_2009_trips = nyc_taxi_challenge_path + 'datasets/data-sample_data-nyctaxi-trips-2009-json_corrigido.json'
trips_df = spark.read.json(json_2009_trips)

trips_df.limit(100).toPandas()

,dropoff_datetime,dropoff_latitude,dropoff_longitude,fare_amount,passenger_count,payment_type,pickup_datetime,pickup_latitude,pickup_longitude,rate_code,store_and_fwd_flag,surcharge,tip_amount,tolls_amount,total_amount,trip_distance,vendor_id
0,2009-04-21T18:57:09.433767+00:00,40.747950,-73.994712,5.4,2,Cash,2009-04-21T18:51:11.767205+00:00,40.742950,-74.004114,None,None,0.0,0.00,0.00,5.40,0.80,CMT
1,2009-01-13T07:50:36.386011+00:00,40.792385,-73.940449,15.4,1,Cash,2009-01-13T07:40:07.639754+00:00,40.747784,-73.996506,None,None,0.0,0.00,0.00,15.40,5.40,CMT
2,2009-01-06T19:43:09.908429+00:00,40.768108,-73.944535,6.1,5,CASH,2009-01-06T19:30:44.142187+00:00,40.752070,-73.951340,None,None,1.0,0.00,0.00,7.10,1.67,VTS
3,2009-09-24T08:41:56.739991+00:00,40.721812,-73.958122,12.9,1,CASH,2009-09-24T08:28:09.004451+00:00,40.729128,-74.001117,None,None,0.5,0.00,0.00,13.40,4.13,VTS
4,2009-10-29T05:57:22.880174+00:00,40.776075,-73.979757,7.7,1,CASH,2009-10-29T05:50:39.214629+00:00,40.756873,-73.976600,None,None,0.5,0.00,0.00,8.20,2.03,VTS
5,2009-10-23T17:23:51.913687+00:00,40.750478,-73.978623,6.5,1,CASH,2009-10-23T17:14:45.661393+00:00,40.750528,-73.994665,None,None,0.0,0.00,0.00,6.50,1.15,VTS
6,2009-03-30T09:47:12.908395+00:00,40.774042,-73.870838,22.1,1,CASH,2009-03-30T09:41:23.359373+00:00,40.772843,-73.955462,None,None,0.0,0.00,4.15,26.25,8.61,VTS
7,2009-03-18T18:02:40.735727+00:00,40.759317,-73.985112,5.7,5,Credit,2009-03-18T17:56:30.217996+00:00,40.757998,-73.976158,None,None,0.0,2.00,0.00,7.70,0.84,VTS
8,2009-03-30T13:32:11.850561+00:00,40.761585,-73.975072,6.1,2,CASH,2009-03-30T13:21:23.577454+00:00,40.764408,-73.988585,None,None,0.0,0.00,0.00,6.10,1.04,VTS
9,2009-02-20T18:11:48.973565+00:00,40.747330,-73.993722,3.7,5,CASH,2009-02-20T18:06:37.232527+00:00,40.740683,-74.001978,None,None,0.0,0.00,0.00,3.70,0.75,VTS


## Payment lookup

This one is of payment lookup's:

In [15]:
from pyspark.sql.functions import col

csv_payment_lookup = nyc_taxi_challenge_path + 'datasets/data-payment_lookup-csv.csv'
payment_lookup_df = (
    spark.read
        .format('csv')
        .option('header', True)
        .load(csv_payment_lookup)
        .filter(col('A') != 'payment_type')
        .withColumnRenamed('A', 'payment_type')
        .withColumnRenamed('B', 'payment_lookup')
)

payment_lookup_df.limit(100).toPandas()

,payment_type,payment_lookup
0,Cas,Cash
1,CAS,Cash
2,Cre,Credit
3,CRE,Credit
4,No,No Charge
5,Dis,Dispute
6,Cash,Cash
7,CASH,Cash
8,Credit,Credit
9,CREDIT,Credit


## Vendor lookup

This is the vendor's lookup dataset:

In [17]:
from pyspark.sql.functions import col

csv_vendor_lookup = nyc_taxi_challenge_path + 'datasets/data-vendor_lookup-csv.csv'
vendor_lookup_df = (
    spark.read
        .format('csv')
        .option('header', True)
        .load(csv_vendor_lookup)
)

vendor_lookup_df.limit(100).toPandas()

,vendor_id,name,address,city,state,zip,country,contact,current
0,CMT,"Creative Mobile Technologies, LLC",950 4th Road Suite 78,Brooklyn,NY,11210,USA,contactCMT@gmail.com,Yes
1,VTS,VeriFone Inc,26 Summit St.,Flushing,NY,11354,USA,admin@vtstaxi.com,Yes
2,DDS,"Dependable Driver Service, Inc",8554 North Homestead St.,Bronx,NY,10472,USA,9778896500,Yes
3,TS,Total Solutions Co,Five Boroughs Taxi Co.,Brooklyn,NY,11229,USA,mgmt@5btc.com,Yes
4,MT,Mega Taxi,4 East Jennings St.,Brooklyn,NY,11228,USA,contact@megataxico.com,No


# Questions

Queremos que, a partir dos dados disponibilizados, você responda, de preferência com gráficos às seguintes questões:

1. Qual a distância média percorrida por viagens com no máximo 2 passageiros;
2. Quais os 3 maiores ​vendors​ em quantidade total de dinheiro arrecadado;
3. Faça um histograma da distribuição mensal, nos 4 anos, de corridas pagas em dinheiro;
4. Faça um gráfico de série temporal contando a quantidade de gorjetas de cada dia, nos últimos 3 meses de 2012.

## Distância média percorrida por viagens com no máximo 2 passageiros

## Os 3 maiores vendors em quantidade total de dinheiro arrecadado

## Histograma da distribuição mensal (nos 4 anos) de corridas pagas em dinheiro

## Quantidade de gorjetas por dia nos últimos 3 meses de 2012

# Bonus questions


- Qual o tempo médio das corridas nos dias de sábado e domingo;
- Fazer uma visualização em mapa com latitude e longitude de ​pickups and ​dropoffs ​no ano de 2010;
- Simular um ​streaming dos dados dos ​JSON e fazer uma visualização acompanhando uma métrica em tempo-real;
- Conseguir provisionar todo seu ambiente em uma cloud pública, de preferência ​AWS​.

## Tempo médio das corridas nos dias de sábado e domingo

## Visualização em mapa com latitude e longitude de pickups e dropoffs no ano de 2010